Wczytanie danych oraz wyświetlenie pierwszych kilku wierszy

In [ ]:
import pandas as pd

df = pd.read_csv('dataset/House_Rent_Dataset.csv')
df.head()

Informacje o danych oraz Statystyki opisowe

In [ ]:
df.info()
df.describe()

Sprawdzenie brakujących wartości

Przykład kodowania one-hot dla kolumny 'Furnishing Status'

In [ ]:
df = pd.get_dummies(df, columns=['Furnishing Status'], drop_first=True)


Kolumna 'Floor' ma format np. '2 out of 5'. Możemy ją podzielić na dwie kolumny: 'Floor Level' i 'Total Floors'.

In [ ]:
def split_floor(x):
    parts = x.split(' out of ')
    floor_level = parts[0]
    total_floors = parts[1] if len(parts) > 1 else None
    return pd.Series([floor_level, total_floors])

df[['Floor Level', 'Total Floors']] = df['Floor'].apply(split_floor)

df.drop('Floor', axis=1, inplace=True)


Zamień wartości tekstowe na numeryczne, np. 'Ground' na 0.

In [ ]:
df['Floor Level'] = df['Floor Level'].replace({'Ground': 0, 'Lower Basement': -1, 'Upper Basement': -2})
df['Floor Level'] = pd.to_numeric(df['Floor Level'], errors='coerce')

df['Total Floors'] = pd.to_numeric(df['Total Floors'], errors='coerce')

Usunięcie jednostki 'sqft' i przekonwertanie jej na typ numeryczny.

In [ ]:
print(df['Size'].dtype)
print(df['Size'].isnull().sum())
df['Size'] = df['Size'].astype(str)
import numpy as np

# Zamień 'nan' na NaN z numpy
df['Size'] = df['Size'].replace('nan', np.nan)

# Usuń ' sqft' i przekonwertuj na float
df['Size'] = df['Size'].str.replace(' sqft', '')
df['Size'] = df['Size'].astype(float)


Przekształcenie kolumny Posted On

In [ ]:
df['Posted On'] = pd.to_datetime(df['Posted On'], format='%Y-%m-%d')

Kolumny takie jak 'Area Locality' i 'Point of Contact' mogą być usunięte

In [ ]:
df.drop(['Area Locality', 'Point of Contact'], axis=1, inplace=True)

Przekształcenie innych kolumn kategorycznych

In [ ]:
categorical_cols = ['Area Type', 'City', 'Tenant Preferred']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

Podział danych na cechy (X) i etykietę (y):

In [ ]:
# Zmienna docelowa
y = df['Rent']

# Cechy
X = df.drop('Rent', axis=1)

Podział danych na zbiory treningowe i testowe:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Zapis pełnego, oczyszczonego zestawu danych

In [ ]:
# Zapis oczyszczonego zestawu danych do pliku CSV
df.to_csv('data/cleaned_data.csv', index=False)

Zapis zbiorów treningowego i testowego

In [ ]:
# Zapis zbioru treningowego
X_train.to_csv('data/X_train.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)

# Zapis zbioru testowego
X_test.to_csv('data/X_test.csv', index=False)
y_test.to_csv('data/y_test.csv', index=False)